In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function, division

from matplotlib import pyplot as plt
import pandas
import src
import gensim
import os
import os.path
import csv
import functools
import itertools
import collections
import scipy
import scipy.stats
from operator import itemgetter
from pprint import pprint

def fake(*args, **kwargs):
    print('Fake called with', str(args), str(kwargs))
    sys.exit(1)

# fake out the create_model so we don't accidentally attempt to create data
src.common.create_model = fake

In [3]:
print(os.getcwd())
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
print(os.getcwd())

/home/cscorley/git/triage/notebooks
/home/cscorley/git/triage


In [4]:
def wilcoxon(x, y):
    T, p = scipy.stats.wilcoxon(x.dropna(), y.dropna(), correction=True)

    nonzeros = sum(1 for a, b in zip(x, y) if (a - b) != 0)
    S = sum(range(1, nonzeros + 1))

    #assert any([item == 0 for item in x]), "x has 0"
    #assert any([item == 0 for item in y]), "y has 0"

    assert S >= T, "%f %f" % (S, T)

    Td = S - T
    rsp1 = Td / S
    rsp2 = T / S
    r = rsp1 - rsp2
    # From this information alone, the remaining rank sum can be computed, because
    # it is the total sum S minus T, or in this case 45 - 18 = 27. Next, the two
    # rank-sum proportions are 27/45 = 60% and 18/45 = 40%. Finally, the rank
    # correlation is the difference between the two proportions (.60 minus .40),
    # hence r = .20.
    return T, p, r

In [5]:
kwargs = dict(model="lda", level="file", source=["changeset"], force=False, rankpath='', config=dict())
default_model_config, model_config_string = src.main.get_default_model_config(kwargs)
default_changeset_config, changeset_config_string = src.main.get_default_changeset_config()

model_config = dict(default_model_config)

model_config.update({
    'alpha_base': 1,
    'eta_base': 1,
})

changeset_config = dict(default_changeset_config)

def get_config_string(config):
    return '-'.join([unicode(v) for k, v in sorted(config.items()) if not k.endswith("_base")])

def get_config_string2(table):
    return "".join(sorted([key.replace('include_', '').upper()[0] for key, value in table.iteritems() if key != "FLT" and key != "DIT" and value]))

alpha_bases = ['auto', 1, 2, 5]
eta_bases = ['auto', 1, 2, 5]
num_topics = [100, 200, 500]

def get_rank_name(kind, experiment, changeset_config, model_config):
    """
    kind = [changeset, release, temporal]
    experiment = [triage, feature_location]
    """
    cs_str = get_config_string(changeset_config)
    model_config = dict(model_config)
    m_str = get_config_string(model_config)
    return '-'.join([kind, experiment, 'lda', cs_str, m_str, 'file', 'ranks']).lower() + '.csv.gz'

model_sweep = list()
for a, e, K in itertools.product(alpha_bases, eta_bases, num_topics):
    m = dict(model_config)
    m['alpha_base'] = a
    m['eta_base'] = e

    if a != 'auto':
        a /= K
    if e != 'auto':
        e /= K
    m['alpha'] = a
    m['eta'] = e
    m['num_topics'] = K
    model_sweep.append(m)

corpus_sweep = list()
b = [True, False]
for a, c, m, r in itertools.product(b, repeat=4):
    conf = dict(changeset_config)
    conf['include_additions'] = a
    conf['include_context'] = c
    conf['include_message'] = m
    conf['include_removals'] = r
    if any(conf.values()):
        corpus_sweep.append(conf)

In [6]:
get_config_string2(corpus_sweep[0])

'ACMR'

In [7]:


# All of the sweep files need renaming throughout.  The *do* use the seed, it's just not named correctly
model_config_string = model_config_string.replace("seed1-", "")

kwargs.update({'changeset_config': changeset_config,
               'changeset_config_string': changeset_config_string})

kwargs.update({'model_config': model_config,
               'model_config_string': model_config_string})

# load project info
projects = src.common.load_projects(kwargs)

#projects = src.common.load_projects(dict(model="lda", level="file", rankpath='', config=dict()), "../data")
projects

[Project(name='tika', printable_name='Tika v1.8', version='v1.8', ref='refs/tags/1.8', data_path='data/tika/', full_path='data/tika/v1.8/', src_path='data/tika/v1.8/src/', model_config={'passes': 1, 'num_topics': 500, 'algorithm': 'batch', 'decay': 0.5, 'eta_base': 1, 'alpha_base': 1, 'eta': 0.002, 'iterations': 1000, 'offset': 1.0, 'alpha': 0.002, 'max_bound_iterations': 1000}, source=['changeset'], model_config_string=u'batch-0.002-0.5-0.002-1000-1000-500-1.0-1', force=False, level='file', changeset_config_string=u'True-True-False-True', model='lda', config={}, changeset_config={'include_removals': True, 'include_message': False, 'include_additions': True, 'include_context': True}, rankpath=''),
 Project(name='pig', printable_name='Pig v0.14.0', version='v0.14.0', ref='refs/tags/release-0.14.0', data_path='data/pig/', full_path='data/pig/v0.14.0/', src_path='data/pig/v0.14.0/src/', model_config={'passes': 1, 'num_topics': 500, 'algorithm': 'batch', 'decay': 0.5, 'eta_base': 1, 'alpha

In [8]:
cs_dit = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'triage', c, model_config)) for c in corpus_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version, rankname)
    if os.path.exists(rankpath):
        #  cs_dit.append(project._replace(rankpath=rankpath, config=config))
        project = project._replace(changeset_config=config, rankpath=rankpath, config=config)
        cs_dit.append(
            project._replace(
                model_config_string=get_config_string(project.model_config),
                changeset_config_string=get_config_string(project.changeset_config)
            ))
    else:
        print(rankpath)

cs_flt = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'feature_location', c, model_config)) for c in corpus_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version,  rankname)
    if os.path.exists(rankpath):
        #cs_flt.append(project._replace(rankpath=rankpath, config=config))
        project = project._replace(changeset_config=config, rankpath=rankpath, config=config)
        cs_flt.append(
            project._replace(
                model_config_string=get_config_string(project.model_config),
                changeset_config_string=get_config_string(project.changeset_config)
            ))
    else:
        print(rankpath)

        
ms_dit = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'triage', changeset_config, c)) for c in model_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version,  rankname)
    if os.path.exists(rankpath):
        #ms_dit.append(project._replace(rankpath=rankpath, config=config))
        project = project._replace(model_config=config, rankpath=rankpath, config=config)
        ms_dit.append(
            project._replace(
                model_config_string=get_config_string(project.model_config),
                changeset_config_string=get_config_string(project.changeset_config)
            ))
    else:
        print(rankpath)
        
ms_flt = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'feature_location', changeset_config, c)) for c in model_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version,  rankname)
    if os.path.exists(rankpath):
        #ms_flt.append(project._replace(rankpath=rankpath, config=config))
        project = project._replace(model_config=config, rankpath=rankpath, config=config)
        ms_flt.append(
            project._replace(
                model_config_string=get_config_string(project.model_config),
                changeset_config_string=get_config_string(project.changeset_config)
            ))
    else:
        print(rankpath)

In [9]:
cs_dit[0]
src.triage.run_experiment(cs_dit[0])

{'changeset': [(4, '1269', u'Nick_Burch_<nick@apache.org>'),
  (6, '1483', u'Chris_Mattmann_<mattmann@apache.org>'),
  (7, '1548', u'Tim_Allison_<tallison@apache.org>'),
  (3, '1489', u'Tim_Allison_<tallison@apache.org>'),
  (9, '1547', u'Tyler_Palsulich_<tpalsulich@apache.org>'),
  (7, '1544', u'Tim_Allison_<tallison@apache.org>'),
  (1, '1589', u'Nick_Burch_<nick@apache.org>'),
  (7, '1542', u'Tim_Allison_<tallison@apache.org>'),
  (3, '1541', u'Chris_Mattmann_<mattmann@apache.org>'),
  (1, '1580', u'Chris_Mattmann_<mattmann@apache.org>'),
  (19, '1581', u'Hong-Thai_Nguyen_<thaichat04@apache.org>'),
  (1, '1587', u'grossws_<grossws@unknown>'),
  (12, '1584', u'Tim_Allison_<tallison@apache.org>'),
  (6, '1549', u'Chris_Mattmann_<mattmann@apache.org>'),
  (1, '1578', u'Ann_Bryant_Burgess_<aburgess@apache.org>'),
  (1, '1521', u'Nick_Burch_<nick@apache.org>'),
  (5, '1561', u'Chris_Mattmann_<mattmann@apache.org>'),
  (3, '1563', u'Nick_Burch_<nick@apache.org>'),
  (4, '936', u'Jukka_Lau

In [10]:
corpus_df = pandas.DataFrame(columns=["Subject", "Task", "Issue", "Rank", "Item", "Additions", "Removals", "Context", "Message"])
for item in cs_dit:
    df = pandas.DataFrame(columns=corpus_df.columns)
    result = src.triage.run_experiment(item)["changeset"]
    df["Rank"] = [x for x, y, z in result]
    df["Issue"] = [y for x, y, z in result]
    df["Item"] = [z for x, y, z in result]
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["Additions"] = df.Additions.fillna(item.config['include_additions'])
    df["Removals"] = df.Removals.fillna(item.config['include_removals'])
    df["Context"] = df.Context.fillna(item.config['include_context'])
    df["Message"] = df.Message.fillna(item.config['include_message'])
    df["Task"] = df.Task.fillna("DIT")
    corpus_df = corpus_df.append(df, ignore_index=True)
    
for item in cs_flt:
    df = pandas.DataFrame(columns=corpus_df.columns)
    result = src.feature_location.run_experiment(item)["changeset"]
    df["Rank"] = [x for x, y, z in result]
    df["Issue"] = [y for x, y, z in result]
    df["Item"] = [z for x, y, z in result]
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["Additions"] = df.Additions.fillna(item.config['include_additions'])
    df["Removals"] = df.Removals.fillna(item.config['include_removals'])
    df["Context"] = df.Context.fillna(item.config['include_context'])
    df["Message"] = df.Message.fillna(item.config['include_message'])
    df["Task"] = df.Task.fillna("FLT")
    corpus_df = corpus_df.append(df, ignore_index=True)

In [11]:
model_df = pandas.DataFrame(columns=["Subject", "Task", "Issue", "Rank", "Item", "alpha", "eta", "K"])
for item in ms_dit:
    df = pandas.DataFrame(columns=model_df.columns)
    result = src.triage.run_experiment(item)["changeset"]
    df["Rank"] = [x for x, y, z in result]
    df["Issue"] = [y for x, y, z in result]
    df["Item"] = [z for x, y, z in result]
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["alpha"] = df.alpha.fillna(item.config['alpha_base'])
    df["eta"] = df.eta.fillna(item.config['eta_base'])
    df["K"] = df.K.fillna(item.config['num_topics'])
    df["Task"] = df.Task.fillna("DIT")
    model_df = model_df.append(df, ignore_index=True)
    
for item in ms_flt:
    df = pandas.DataFrame(columns=model_df.columns)
    result = src.feature_location.run_experiment(item)["changeset"]
    df["Rank"] = [x for x, y, z in result]
    df["Issue"] = [y for x, y, z in result]
    df["Item"] = [z for x, y, z in result]
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["alpha"] = df.alpha.fillna(item.config['alpha_base'])
    df["eta"] = df.eta.fillna(item.config['eta_base'])
    df["K"] = df.K.fillna(item.config['num_topics'])
    df["Task"] = df.Task.fillna("FLT")
    model_df = model_df.append(df, ignore_index=True)

# Corpus analysis

In [12]:
len(corpus_sweep)

15

#### These are inequal because of the False, False, False, False configuration being invalid.

In [13]:
corpus_df.groupby(["Subject", "Task", "Additions"]).Issue.apply(len)

Subject            Task  Additions
BookKeeper v4.3.0  DIT   False        1148
                         True         1312
                   FLT   False        1001
                         True         1144
Mahout v0.10.0     DIT   False         931
                         True         1064
                   FLT   False         350
                         True          400
OpenJPA v2.3.0     DIT   False         959
                         True         1096
                   FLT   False         917
                         True         1048
Pig v0.14.0        DIT   False        1554
                         True         1776
                   FLT   False        1218
                         True         1392
Tika v1.8          DIT   False         280
                         True          320
                   FLT   False         252
                         True          288
ZooKeeper v3.5.0   DIT   False        2513
                         True         2872
                   

In [14]:
(corpus_df.groupby(["Task", "Additions"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Task", "Removals"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Task", "Context"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Task", "Message"]).Rank.apply(src.utils.calculate_mrr),
)

(Task  Additions
 DIT   False        0.355573
       True         0.364590
 FLT   False        0.361306
       True         0.418083
 Name: Rank, dtype: float64,
 '********************************************',
 Task  Removals
 DIT   False       0.373944
       True        0.348516
 FLT   False       0.398596
       True        0.385454
 Name: Rank, dtype: float64,
 '********************************************',
 Task  Context
 DIT   False      0.340561
       True       0.377725
 FLT   False      0.385554
       True       0.396867
 Name: Rank, dtype: float64,
 '********************************************',
 Task  Message
 DIT   False      0.358320
       True       0.362187
 FLT   False      0.376123
       True       0.405118
 Name: Rank, dtype: float64)

In [15]:
(corpus_df.groupby(["Subject", "Task", "Additions"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Subject", "Task", "Removals"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Subject", "Task", "Context"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Subject", "Task", "Message"]).Rank.apply(src.utils.calculate_mrr),
)

(Subject            Task  Additions
 BookKeeper v4.3.0  DIT   False        0.646312
                          True         0.633168
                    FLT   False        0.402220
                          True         0.500213
 Mahout v0.10.0     DIT   False        0.291689
                          True         0.319166
                    FLT   False        0.291606
                          True         0.288826
 OpenJPA v2.3.0     DIT   False        0.311652
                          True         0.372377
                    FLT   False        0.263736
                          True         0.311058
 Pig v0.14.0        DIT   False        0.214010
                          True         0.169551
                    FLT   False        0.321887
                          True         0.379533
 Tika v1.8          DIT   False        0.350845
                          True         0.393482
                    FLT   False        0.435476
                          True         0.493537
 Zoo

In [16]:
# what the fuck was I doing here?
res = pandas.DataFrame(columns=["Subject", "Task", "Config", "ExcludeMRR", "IncludeMRR", "p"])
for k in ["Additions", "Removals", "Context", "Message"]:
    for key, group in corpus_df.groupby(["Subject", "Task"]):
        subject, task = key
        sub = group.groupby(k).groups
        f = corpus_df.ix[sub[False]].Rank
        t = corpus_df.ix[sub[True]].Rank
        stat, p = scipy.stats.mannwhitneyu(f, t)
        # Must use Mann Whitney here instead of Wilcoxon because the FFFF config (all False) creates an offset in the total number of ranks!
        
        res = res.append(
            dict(
                zip(res.columns, 
                    [subject, task, k, src.utils.calculate_mrr(f), src.utils.calculate_mrr(t), p]))
            , ignore_index=True)
    sub = corpus_df.groupby(["Task", k]).groups
    f = corpus_df.ix[sub[("DIT", False)]].Rank
    t = corpus_df.ix[sub[("DIT", True)]].Rank
    stat, p = scipy.stats.mannwhitneyu(f, t)
    res = res.append(
        dict(
            zip(res.columns, 
                ["Overall", "DIT", k, src.utils.calculate_mrr(f), src.utils.calculate_mrr(t), p]))
        , ignore_index=True)
    f = corpus_df.ix[sub[("FLT", False)]].Rank
    t = corpus_df.ix[sub[("FLT", True)]].Rank
    stat, p = scipy.stats.mannwhitneyu(f, t)
    res = res.append(
        dict(
            zip(res.columns, 
                ["Overall", "FLT", k, src.utils.calculate_mrr(f), src.utils.calculate_mrr(t), p]))
        , ignore_index=True)        

In [17]:
res[(res.ExcludeMRR > res.IncludeMRR) & (res.p < 0.01)]

,Subject,Task,Config,ExcludeMRR,IncludeMRR,p
16,Mahout v0.10.0,DIT,Removals,0.327144,0.288143,8.426824e-04
20,Pig v0.14.0,DIT,Removals,0.220149,0.164180,2.857885e-12
24,ZooKeeper v3.5.0,DIT,Removals,0.378937,0.348919,1.159353e-08
26,Overall,DIT,Removals,0.373944,0.348516,3.629592e-13
27,Overall,FLT,Removals,0.398596,0.385454,1.726129e-03
52,ZooKeeper v3.5.0,DIT,Message,0.372189,0.354824,8.063377e-04


In [18]:
res[(res.ExcludeMRR <= res.IncludeMRR) & (res.p < 0.01)]

,Subject,Task,Config,ExcludeMRR,IncludeMRR,p
1,BookKeeper v4.3.0,FLT,Additions,0.402220,0.500213,1.223079e-09
2,Mahout v0.10.0,DIT,Additions,0.291689,0.319166,5.049910e-03
4,OpenJPA v2.3.0,DIT,Additions,0.311652,0.372377,3.547040e-06
7,Pig v0.14.0,FLT,Additions,0.321887,0.379533,7.796939e-05
10,ZooKeeper v3.5.0,DIT,Additions,0.351252,0.373143,1.506603e-09
11,ZooKeeper v3.5.0,FLT,Additions,0.421908,0.470905,8.444801e-05
12,Overall,DIT,Additions,0.355573,0.364590,6.921688e-04
13,Overall,FLT,Additions,0.361306,0.418083,6.912758e-14
28,BookKeeper v4.3.0,DIT,Context,0.592102,0.680602,1.904592e-10
34,Pig v0.14.0,DIT,Context,0.171824,0.206464,9.967250e-42


In [19]:
# for key, group in corpus_df.groupby(["Subject", "Task"]):
#     ranks = dict()
#     for subkey, subgroup in group.groupby(["Additions", "Removals", "Context", "Message"]):
#         ranks[subkey] = subgroup.Rank

#     print(key, scipy.stats.friedmanchisquare(*ranks.values()))
#     for x, y in itertools.combinations(corpus_df.groupby(["Additions", "Removals", "Context", "Message"]).groups.keys(), r=2):
#         stat, p = scipy.stats.wilcoxon(ranks[x], ranks[y])
#         if p < 0.01:
#             print(x, y, p, "******")
#         else:
#             print(x, y, p)
#     print()

In [20]:
cres = pandas.DataFrame(columns=["Subject", "Task", "Config", "Config2", "MRR", "MRR2", "T", "p", "r"])

for key, group in corpus_df.groupby(["Subject", "Task"]):
    for c1, c2 in itertools.combinations(corpus_sweep, r=2):
        a = group[
            (group.Additions == c1["include_additions"]) &
            (group.Removals == c1["include_removals"]) &
            (group.Context == c1["include_context"]) &
            (group.Message == c1["include_message"])
                 ].Rank
        b = group[
            (group.Additions == c2["include_additions"]) &
            (group.Removals == c2["include_removals"]) &
            (group.Context == c2["include_context"]) &
            (group.Message == c2["include_message"])
                 ].Rank
        stat, p, r = wilcoxon(a,b)
        cres = cres.append(
            dict(
                zip(cres.columns, 
                    [key[0], key[1],
                     get_config_string2(c1),
                     get_config_string2(c2),
                     src.utils.calculate_mrr(a), 
                     src.utils.calculate_mrr(b),
                     stat, p,r]))
            , ignore_index=True)

for key, group in corpus_df.groupby(["Task"]):
    for c1, c2 in itertools.combinations(corpus_sweep, r=2):
        a = group[
            (group.Additions == c1["include_additions"]) &
            (group.Removals == c1["include_removals"]) &
            (group.Context == c1["include_context"]) &
            (group.Message == c1["include_message"])
                 ].Rank
        b = group[
            (group.Additions == c2["include_additions"]) &
            (group.Removals == c2["include_removals"]) &
            (group.Context == c2["include_context"]) &
            (group.Message == c2["include_message"])
                 ].Rank
        stat, p, r = wilcoxon(a,b)
        cres = cres.append(
            dict(
                zip(cres.columns, 
                    ["all subject systems", key,
                     get_config_string2(c1),
                     get_config_string2(c2),
                     src.utils.calculate_mrr(a), 
                     src.utils.calculate_mrr(b),
                     stat, p,r]))
            , ignore_index=True)


In [21]:
bon=0.01/len(list(itertools.combinations(range(15),2)))
bon, len(cres[(cres.p < bon)]), len(cres[(cres.p < (0.01))]), len(cres)
bon = 0.01

In [22]:
len(list(itertools.combinations(range(15),2)))

105

In [23]:
d = cres[(cres.Subject == "all subject systems") &
         (cres.Task == "DIT") &
         (cres.p < bon)]
len(d), d

(86,
                   Subject Task Config Config2       MRR      MRR2         T  \
 1260  all subject systems  DIT   ACMR     ACM  0.378414  0.378506  113729.5   
 1263  all subject systems  DIT   ACMR     AMR  0.378414  0.364607  110678.5   
 1264  all subject systems  DIT   ACMR      AM  0.378414  0.353739   94705.5   
 1265  all subject systems  DIT   ACMR      AR  0.378414  0.346219  119198.0   
 1266  all subject systems  DIT   ACMR       A  0.378414  0.337990  122665.0   
 1267  all subject systems  DIT   ACMR     CMR  0.378414  0.339059   98301.5   
 1268  all subject systems  DIT   ACMR      CM  0.378414  0.416467  132472.5   
 1269  all subject systems  DIT   ACMR      CR  0.378414  0.337312  106166.0   
 1270  all subject systems  DIT   ACMR       C  0.378414  0.414800  139566.0   
 1271  all subject systems  DIT   ACMR      MR  0.378414  0.330810   84658.0   
 1272  all subject systems  DIT   ACMR       M  0.378414  0.335894  156367.5   
 1273  all subject systems  DIT   A

In [24]:
f = cres[(cres.Subject == "all subject systems") &
         (cres.Task == "FLT") &
         (cres.p < bon)]
len(f), f

(49,
                   Subject Task Config Config2       MRR      MRR2        T  \
 1372  all subject systems  FLT   ACMR     CMR  0.431519  0.401344  82302.5   
 1374  all subject systems  FLT   ACMR      CR  0.431519  0.344348  75774.0   
 1375  all subject systems  FLT   ACMR       C  0.431519  0.338609  87610.5   
 1376  all subject systems  FLT   ACMR      MR  0.431519  0.348768  80121.0   
 1378  all subject systems  FLT   ACMR       R  0.431519  0.315149  74713.5   
 1385  all subject systems  FLT    ACM     CMR  0.451749  0.401344  78164.5   
 1386  all subject systems  FLT    ACM      CM  0.451749  0.397141  82657.5   
 1387  all subject systems  FLT    ACM      CR  0.451749  0.344348  72468.5   
 1388  all subject systems  FLT    ACM       C  0.451749  0.338609  77795.5   
 1389  all subject systems  FLT    ACM      MR  0.451749  0.348768  74925.0   
 1391  all subject systems  FLT    ACM       R  0.451749  0.315149  68396.5   
 1399  all subject systems  FLT    ACR      CR 

In [25]:
friedman_df = pandas.DataFrame(columns=["Subject", "Task", "$\chi^2(15)$", "p-value", "Post-hoc Wilcoxon"])

for key, group in corpus_df.groupby(["Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["Additions", "Removals", "Context", "Message"]):
        assert subkey not in ranks
        ranks[subkey] = subgroup.Rank
        
    post_hoc = len(cres[(cres.Subject == "all subject systems") &
                        (cres.Task == key) &
                        (cres.p < bon)])

    stat, p = scipy.stats.friedmanchisquare(*ranks.values())
    friedman_df = friedman_df.append(
        dict(
            zip(friedman_df.columns, 
                ["all subject systems", key, stat, p, post_hoc]))
        , ignore_index=True)

for key, group in corpus_df.groupby(["Subject", "Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["Additions", "Removals", "Context", "Message"]):
        assert subkey not in ranks
        ranks[subkey] = subgroup.Rank
        
    post_hoc = len(cres[(cres.Subject == key[0]) &
                        (cres.Task == key[1]) &
                        (cres.p < bon)])
        
    stat, p = scipy.stats.friedmanchisquare(*ranks.values())
    friedman_df = friedman_df.append(
        dict(
            zip(friedman_df.columns, 
                [key[0], key[1], stat, p, post_hoc]))
        , ignore_index=True)
    
friedman_df

,Subject,Task,$\chi^2(15)$,p-value,Post-hoc Wilcoxon
0,all subject systems,DIT,1026.420224,3.351323e-210,86
1,all subject systems,FLT,204.925199,5.408174e-36,49
2,BookKeeper v4.3.0,DIT,269.159657,3.082638e-49,57
3,BookKeeper v4.3.0,FLT,120.178710,5.805075e-19,37
4,Mahout v0.10.0,DIT,50.551436,4.935036e-06,23
5,Mahout v0.10.0,FLT,13.595833,4.802349e-01,0
6,OpenJPA v2.3.0,DIT,124.629893,7.768937e-20,42
7,OpenJPA v2.3.0,FLT,39.360167,3.207992e-04,8
8,Pig v0.14.0,DIT,887.094000,2.512999e-180,85
9,Pig v0.14.0,FLT,53.369310,1.645994e-06,12


In [26]:
FIG_TEX = """\\begin{table}
\\begin{spacing}{1.2}
\\centering
\\caption{Friedman test results for %s corpus configuration sweeps. For each system, 105 post-hoc Wilcoxon tests were conducted.}
\\label{table:combo-friedman-%s}
%s
\\end{spacing}
\\end{table}
"""

dit_friedman = friedman_df[friedman_df.Task == "DIT"]
del dit_friedman["Task"]

flt_friedman = friedman_df[friedman_df.Task == "FLT"]
del flt_friedman["Task"]

formatters = {
    "p-value": lambda x: ("$p<%f" % bon).rstrip("0") + "$" if x < bon else "$%.4f$" % x ,
    "Subject": lambda x: x.replace("all", "\\midrule\nAll") if x.startswith("all") else x,
    "$\chi^2(15)$": lambda x: "$%.4f$" % x,
    "Optimal MRR": lambda x: "$%.4f$" % x,
    "Alternate MRR": lambda x: "$%.4f$" % x,
    "Effect size": lambda x: "$%.4f$" % x,
    "Post-hoc Wilcoxon": lambda x: "$%d\;(%.1f\\%%)$" % (x, (float(x)/105.0) * 100),
}

flt_tex = flt_friedman.sort("Subject").to_latex(index=False,
                                            escape=False, # needed so it doesn't screw up formatters
                                            formatters=formatters)

dit_tex = dit_friedman.sort("Subject").to_latex(index=False,
                                            escape=False, # needed so it doesn't screw up formatters
                                            formatters=formatters)

print(FIG_TEX % ("FLT", "flt", flt_tex))
print(FIG_TEX % ("DIT", "dit", dit_tex))
with open(os.path.expanduser("~/git/dissertation/tables/combo_friedman_results.tex"), "wt") as f:
    print(FIG_TEX % ("FLT", "flt", flt_tex), file=f)
    print(FIG_TEX % ("DIT", "dit", dit_tex), file=f)

\begin{table}
\begin{spacing}{1.2}
\centering
\caption{Friedman test results for FLT corpus configuration sweeps. For each system, 105 post-hoc Wilcoxon tests were conducted.}
\label{table:combo-friedman-flt}
\begin{tabular}{lrrr}
\toprule
                      Subject & $\chi^2(15)$ &  p-value & Post-hoc Wilcoxon \\
\midrule
            BookKeeper v4.3.0 &   $120.1787$ & $p<0.01$ &    $37\;(35.2\%)$ \\
               Mahout v0.10.0 &    $13.5958$ & $0.4802$ &      $0\;(0.0\%)$ \\
               OpenJPA v2.3.0 &    $39.3602$ & $p<0.01$ &      $8\;(7.6\%)$ \\
                  Pig v0.14.0 &    $53.3693$ & $p<0.01$ &    $12\;(11.4\%)$ \\
                    Tika v1.8 &    $39.1413$ & $p<0.01$ &      $9\;(8.6\%)$ \\
             ZooKeeper v3.5.0 &    $71.6131$ & $p<0.01$ &    $26\;(24.8\%)$ \\
 \midrule
All subject systems &   $204.9252$ & $p<0.01$ &    $49\;(46.7\%)$ \\
\bottomrule
\end{tabular}

\end{spacing}
\end{table}

\begin{table}
\begin{spacing}{1.2}
\centering
\caption{Friedman t

# Model analysis

In [27]:
model_df[:10]

,Subject,Task,Issue,Rank,Item,alpha,eta,K
0,Tika v1.8,DIT,1269,9,Tyler_Palsulich_<tpalsulich@apache.org>,auto,auto,100
1,Tika v1.8,DIT,1483,15,Chris_Mattmann_<mattmann@apache.org>,auto,auto,100
2,Tika v1.8,DIT,1548,5,Tim_Allison_<tallison@apache.org>,auto,auto,100
3,Tika v1.8,DIT,1489,4,Tim_Allison_<tallison@apache.org>,auto,auto,100
4,Tika v1.8,DIT,1547,4,Tyler_Palsulich_<tpalsulich@apache.org>,auto,auto,100
5,Tika v1.8,DIT,1544,3,Tim_Allison_<tallison@apache.org>,auto,auto,100
6,Tika v1.8,DIT,1589,1,Nick_Burch_<nick@apache.org>,auto,auto,100
7,Tika v1.8,DIT,1542,9,Tim_Allison_<tallison@apache.org>,auto,auto,100
8,Tika v1.8,DIT,1541,11,Chris_Mattmann_<mattmann@apache.org>,auto,auto,100
9,Tika v1.8,DIT,1580,1,Chris_Mattmann_<mattmann@apache.org>,auto,auto,100


In [28]:
# for key, group in model_df.groupby(["Subject", "Task"]):
#     ranks = dict()
#     for subkey, subgroup in group.groupby(["alpha", "eta", "K"]):
#         ranks[subkey] = subgroup.Rank

#     print(key, scipy.stats.friedmanchisquare(*ranks.values()))
#     for x, y in itertools.combinations(model_df.groupby(["alpha", "eta", "K"]).groups.keys(), r=2):
#         stat, p = scipy.stats.wilcoxon(ranks[x], ranks[y])
#         if p < 0.01:
#             print(x, y, p, "******")
#         else:
#             print(x, y, p)
#     print()

In [29]:
for key, group in model_df.groupby(["Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["alpha", "eta", "K"]):
        assert subkey not in ranks
        ranks[subkey] = subgroup.Rank

    print(key, scipy.stats.friedmanchisquare(*ranks.values()))

print('----')

for key, group in model_df.groupby(["Subject", "Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["alpha", "eta", "K"]):
        assert subkey not in ranks
        ranks[subkey] = subgroup.Rank

    print(key, scipy.stats.friedmanchisquare(*ranks.values()))

DIT (2765.6842770138119, 0.0)
FLT (1268.1662872188358, 9.0338652843896644e-235)
----
('BookKeeper v4.3.0', 'DIT') (914.49405131140975, 3.6823690924705265e-161)
('BookKeeper v4.3.0', 'FLT') (127.67039786455341, 2.2117762416697342e-09)
('Mahout v0.10.0', 'DIT') (384.55290252857844, 1.6150248713121185e-54)
('Mahout v0.10.0', 'FLT') (150.24132992327392, 1.0048201595768155e-12)
('OpenJPA v2.3.0', 'DIT') (184.24751696601817, 3.8012496245453142e-18)
('OpenJPA v2.3.0', 'FLT') (381.52881843862752, 6.1398023875961168e-54)
('Pig v0.14.0', 'DIT') (354.7128566365742, 8.0054132695147136e-49)
('Pig v0.14.0', 'FLT') (399.58170379169212, 2.0758617659852637e-57)
('Tika v1.8', 'DIT') (184.08385594624323, 4.0448423099902757e-18)
('Tika v1.8', 'FLT') (113.12362651907993, 2.2419304091526999e-07)
('ZooKeeper v3.5.0', 'DIT') (1938.788023570414, 0.0)
('ZooKeeper v3.5.0', 'FLT') (313.26915032741539, 4.9771759841601103e-41)


In [30]:
res = pandas.DataFrame(columns=["Subject", "Task", "Config", "Config2", "MRR", "MRR2", "p"])
for k in ["alpha", "eta", "K"]:
    for key, group in model_df.groupby(["Subject", "Task"]):
        ranks = dict()
        for subkey, subgroup in group.groupby(k):
            assert subkey not in ranks
            ranks[subkey] = subgroup.Rank
        
        for each in itertools.combinations(ranks.keys(), r=2):
            f, t = each
            stat, p = scipy.stats.wilcoxon(ranks[f], ranks[t], correction=True)
            res = res.append(
                dict(
                    zip(res.columns, 
                        [key[0], key[1], k + "=" + str(f), k + "=" + str(t), src.utils.calculate_mrr(ranks[f]), src.utils.calculate_mrr(ranks[t]), p]))
                , ignore_index=True)

ores = pandas.DataFrame(columns=["Subject", "Task", "Config", "Config2", "MRR", "MRR2", "p"])
for k in ["alpha", "eta", "K"]:
    for key, group in model_df.groupby(["Task"]):
        ranks = dict()
        for subkey, subgroup in group.groupby(k):
            assert subkey not in ranks
            ranks[subkey] = subgroup.Rank
        
        for each in itertools.combinations(ranks.keys(), r=2):
            f, t = each
            stat, p = scipy.stats.wilcoxon(ranks[f], ranks[t], correction=True)
            ores = ores.append(
                dict(
                    zip(ores.columns, 
                        ["Overall", key, k + "=" + str(f), k + "=" + str(t), src.utils.calculate_mrr(ranks[f]), src.utils.calculate_mrr(ranks[t]), p]))
                , ignore_index=True)

In [31]:
len(res[res.p < 0.01]), len(res[res.p >= 0.01])

(113, 67)

In [32]:
len(ores[ores.p < 0.01]), len(ores[ores.p >= 0.01])

(21, 9)

In [33]:
ores

,Subject,Task,Config,Config2,MRR,MRR2,p
0,Overall,DIT,alpha=1,alpha=2,0.334535,0.333357,1.864625e-01
1,Overall,DIT,alpha=1,alpha=5,0.334535,0.337464,1.804564e-01
2,Overall,DIT,alpha=1,alpha=auto,0.334535,0.334525,5.330456e-02
3,Overall,DIT,alpha=2,alpha=5,0.333357,0.337464,9.951688e-01
4,Overall,DIT,alpha=2,alpha=auto,0.333357,0.334525,1.806949e-02
5,Overall,DIT,alpha=5,alpha=auto,0.337464,0.334525,3.322544e-02
6,Overall,FLT,alpha=1,alpha=2,0.362646,0.362229,8.218030e-17
7,Overall,FLT,alpha=1,alpha=5,0.362646,0.359742,1.432597e-32
8,Overall,FLT,alpha=1,alpha=auto,0.362646,0.362723,2.996998e-14
9,Overall,FLT,alpha=2,alpha=5,0.362229,0.359742,5.572488e-26


In [34]:
res[(res.MRR > res.MRR2) & (res.p < 0.01)]

,Subject,Task,Config,Config2,MRR,MRR2,p
5,BookKeeper v4.3.0,DIT,alpha=5,alpha=auto,0.570063,0.556589,2.943637e-08
6,BookKeeper v4.3.0,FLT,alpha=1,alpha=2,0.435806,0.433651,3.515726e-04
8,BookKeeper v4.3.0,FLT,alpha=1,alpha=auto,0.435806,0.434269,2.587723e-03
11,BookKeeper v4.3.0,FLT,alpha=5,alpha=auto,0.435948,0.434269,1.299403e-09
12,Mahout v0.10.0,DIT,alpha=1,alpha=2,0.294484,0.289548,1.208079e-03
13,Mahout v0.10.0,DIT,alpha=1,alpha=5,0.294484,0.291443,7.416468e-08
19,Mahout v0.10.0,FLT,alpha=1,alpha=5,0.277402,0.260541,2.482687e-09
21,Mahout v0.10.0,FLT,alpha=2,alpha=5,0.285476,0.260541,7.583579e-07
22,Mahout v0.10.0,FLT,alpha=2,alpha=auto,0.285476,0.276755,6.389131e-07
40,Pig v0.14.0,DIT,alpha=2,alpha=auto,0.186871,0.185495,3.795334e-18


In [35]:
res[(res.MRR < res.MRR2) & (res.p < 0.01)]

,Subject,Task,Config,Config2,MRR,MRR2,p
1,BookKeeper v4.3.0,DIT,alpha=1,alpha=5,0.555225,0.570063,1.479346e-08
3,BookKeeper v4.3.0,DIT,alpha=2,alpha=5,0.557563,0.570063,1.235308e-07
7,BookKeeper v4.3.0,FLT,alpha=1,alpha=5,0.435806,0.435948,4.770280e-11
9,BookKeeper v4.3.0,FLT,alpha=2,alpha=5,0.433651,0.435948,1.672122e-08
10,BookKeeper v4.3.0,FLT,alpha=2,alpha=auto,0.433651,0.434269,1.676760e-03
15,Mahout v0.10.0,DIT,alpha=2,alpha=5,0.289548,0.291443,1.057157e-04
16,Mahout v0.10.0,DIT,alpha=2,alpha=auto,0.289548,0.294620,8.291831e-04
17,Mahout v0.10.0,DIT,alpha=5,alpha=auto,0.291443,0.294620,5.302349e-08
18,Mahout v0.10.0,FLT,alpha=1,alpha=2,0.277402,0.285476,2.701432e-07
23,Mahout v0.10.0,FLT,alpha=5,alpha=auto,0.260541,0.276755,8.730452e-09


In [36]:
t = res[(res.Config == "alpha=1") | (res.Config2 == "alpha=1")]
t

,Subject,Task,Config,Config2,MRR,MRR2,p
0,BookKeeper v4.3.0,DIT,alpha=1,alpha=2,0.555225,0.557563,2.051069e-01
1,BookKeeper v4.3.0,DIT,alpha=1,alpha=5,0.555225,0.570063,1.479346e-08
2,BookKeeper v4.3.0,DIT,alpha=1,alpha=auto,0.555225,0.556589,4.486721e-01
6,BookKeeper v4.3.0,FLT,alpha=1,alpha=2,0.435806,0.433651,3.515726e-04
7,BookKeeper v4.3.0,FLT,alpha=1,alpha=5,0.435806,0.435948,4.770280e-11
8,BookKeeper v4.3.0,FLT,alpha=1,alpha=auto,0.435806,0.434269,2.587723e-03
12,Mahout v0.10.0,DIT,alpha=1,alpha=2,0.294484,0.289548,1.208079e-03
13,Mahout v0.10.0,DIT,alpha=1,alpha=5,0.294484,0.291443,7.416468e-08
14,Mahout v0.10.0,DIT,alpha=1,alpha=auto,0.294484,0.294620,4.254184e-01
18,Mahout v0.10.0,FLT,alpha=1,alpha=2,0.277402,0.285476,2.701432e-07


In [37]:
len(t), len(t[t.MRR > t.MRR2]), len(t[t.p < 0.05])

(36, 18, 23)

# table building

In [38]:
model_df[model_df.Rank == 0]

,Subject,Task,Issue,Rank,Item,alpha,eta,K


In [39]:
model_all = model_df.groupby(["Task", "alpha", "eta", "K"]).Rank.apply(lambda x: numpy.mean(1/x))
model_all

Task  alpha  eta   K  
DIT   1      1     100    0.301700
                   200    0.350912
                   500    0.377034
             2     100    0.292285
                   200    0.333820
                   500    0.376133
             5     100    0.285501
                   200    0.310650
                   500    0.365735
             auto  100    0.303069
                   200    0.343411
                   500    0.374172
      2      1     100    0.300678
                   200    0.346168
                   500    0.371791
             2     100    0.291940
                   200    0.328208
                   500    0.380202
             5     100    0.284913
                   200    0.314315
                   500    0.369073
             auto  100    0.302577
                   200    0.338648
                   500    0.371773
      5      1     100    0.315005
                   200    0.345699
                   500    0.379527
             2     100    0.2914

In [40]:
corpus_all = corpus_df.groupby(["Task", "Additions", "Removals", "Context", "Message"]).Rank.apply(lambda x: numpy.mean(1/x))
corpus_all

Task  Additions  Removals  Context  Message
DIT   False      False     False    True       0.335894
                           True     False      0.414800
                                    True       0.416467
                 True      False    False      0.314670
                                    True       0.330810
                           True     False      0.337312
                                    True       0.339059
      True       False     False    False      0.337990
                                    True       0.353739
                           True     False      0.380211
                                    True       0.378506
                 True      False    False      0.346219
                                    True       0.364607
                           True     False      0.377034
                                    True       0.378414
FLT   False      False     False    True       0.383786
                           True     False      0.338609
    

In [41]:
names = {'model': {'score': 'score',
                   'model_base_alpha': 'alpha',
                   'model_base_eta': 'eta',
                   'num_topics': 'K'
                  },
         'corpus': {'score': 'score',
                    'changeset_include_additions': 'Additions',
                    'changeset_include_context': 'Context',
                    'changeset_include_message': 'Message',
                    'changeset_include_removals': 'Removals',
                    },
        }
exps = ['triage', 'feature_location']
table_headers = {
    'model':['K', 'alpha', 'eta', 'FLT', 'DIT'],  # ['Configuration', 'FLT', 'DIT'], 
    'corpus': ['Configuration', 'FLT', 'DIT']
}
groups = {
    'model': ['K', 'alpha', 'eta'],
    'corpus': ['Additions', 'Removals', 'Context', 'Message']
}

full_tex = r"""
\begin{table}
\begin{spacing}{1.2}
\centering
\caption{MRR values of %s %s construction sweep}
\label{table:%s}
\vspace{0.2em}
%s
\end{spacing}
\end{table}
"""

tex_dir = os.path.expanduser("~/git/dissertation/tables")


def do_great_table(main_df, label_name, rq, caption):
    include_fmt = lambda x: "Included" if x else ""
    main_df = pandas.DataFrame(main_df)

    formatters = {
        'FLT': lambda x: r"$\bm{%.4f}$" % x if x == max(main_df["FLT"]) else "$%.4f$" % x,
        'alpha': lambda x: "$%s/K$" % x if x != 'auto' else x,
        'eta': lambda x: "$%s/K$" % x if x != 'auto' else x,
        'K': lambda x: "$%s$" % int(x),
        'Additions': include_fmt,
        'Removals': include_fmt,
        'Context': include_fmt,
        'Message': include_fmt,
        'DIT': lambda x:  r"$\bm{%.4f}$" % x if x == max(main_df["DIT"]) else "$%.4f$" % x,
    }

    # filter out uninteresting rows, like there was no corpus
    main_df = main_df[(main_df["FLT"] != 0) | (main_df["DIT"] != 0)]
    if rq == "model":
        main_df = main_df.sort(["K", "alpha", "eta"])
        """main_df["Configuration"] = (main_df.K.apply(lambda x: "$(K=%s" % int(x)) + 
                                    main_df.alpha.apply(lambda x: r",\alpha=%s/K" % x if x != 'auto' else r',\alpha=auto') + 
                                    main_df.eta.apply(lambda x: r",\eta=%s/K)$" % x if x != 'auto' else r',\eta=auto)$'))"""
        
    else:
        main_df = main_df.sort(["Additions", "Removals", "Context", "Message"], ascending=False)
        main_df["Configuration"] = (main_df.Additions.apply(lambda x: 'A' if x else '') + 
                                    main_df.Removals.apply(lambda x: 'R' if x else '') + 
                                    main_df.Context.apply(lambda x: 'C' if x else '') + 
                                    main_df.Message.apply(lambda x: 'M' if x else ''))
        formatters["Configuration"] = lambda x: "$(%s)$" % ','.join(x)
        
        

    label = "%s_%s_sweep" % (label_name, rq)
    op = os.path.join(tex_dir, label + ".tex")
    
            
    best_flt = main_df[main_df["FLT"] == main_df["FLT"].max()]
    best_dit = main_df[main_df["DIT"] == main_df["DIT"].max()]

    if len(main_df) > 24:
        tex = r"\parbox{.45\linewidth}{\centering %s} \hfill \parbox{.45\linewidth}{\centering %s}"
        mid = len(main_df)//2
        tex = tex % (main_df[:mid].to_latex(index=False,
                                            escape=False, # needed so it doesn't screw up formatters
                                            formatters=formatters,
                                            columns=table_headers[rq]),
                     main_df[mid:].to_latex(index=False,
                                            escape=False, # needed so it doesn't screw up formatters
                                            formatters=formatters,
                                            columns=table_headers[rq]))
    else:
        tex = main_df.to_latex(index=False,
                               escape=False, # needed so it doesn't screw up formatters
                               formatters=formatters,
                               columns=table_headers[rq],)

    # and now the lazy
    this_full_tex = full_tex % (caption, rq, label, tex)
    this_full_tex = this_full_tex.replace(" alpha ", r" $\alpha$ ")
    this_full_tex = this_full_tex.replace(" eta ", r" $\eta$ ")
    this_full_tex = this_full_tex.replace(r"\begin{tabular}{rllrr}", r"\begin{tabular}{rll|rr}")
    this_full_tex = this_full_tex.replace(r"\begin{tabular}{llllrr}", r"\begin{tabular}{llll|rr}")
    this_full_tex = this_full_tex.replace(r"$500$ &  $1/K$ &  $1/K$ &", r"\myrowcolor $500$ &  $1/K$ &  $1/K$ &")
    this_full_tex = this_full_tex.replace(r"Included &  Included &  Included &           &", r"\myrowcolor Included &  Included &  Included &           &")

    #print("Writing to: %s\n%s\n" % (op, this_full_tex))
    print("Writing to:", op)
    with open(op, 'wt') as f:
        f.write(this_full_tex)
        
    return best_dit, best_flt

best_dits = dict({"model": dict(), "corpus": dict()})
best_flts = dict({"model": dict(), "corpus": dict()})

for rq, main_df in [("model", model_all), ("corpus", corpus_all)]:
    names[rq]['score'] = 'score'
    main_df = main_df.unstack(0).reset_index()
    
    best_dit, best_flt = do_great_table(main_df, "all", rq, "all subject systems")
    best_dits[rq]["all subject systems"] = best_dit
    best_flts[rq]["all subject systems"] = best_flt
    
for rq, main_df in [("model", model_df), ("corpus", corpus_df)]:
    names[rq]['score'] = 'score'
    group = groups[rq]
    zz = main_df.groupby(["Subject", "Task"] + group).Rank.apply(lambda x: numpy.mean(1/x))

    for each in zz.index.levels[0]:
        each_df = zz[each].unstack(0).reset_index()
        bad_person = each.split()[0].lower()
        best_dit, best_flt = do_great_table(each_df, bad_person, rq, "\\" + bad_person)
        best_dits[rq][each] = best_dit
        best_flts[rq][each] = best_flt

/home/cscorley/envs/triage/lib/python2.7/site-packages/pandas/core/index.py:4072: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


Writing to: /home/cscorley/git/dissertation/tables/all_model_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/all_corpus_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/bookkeeper_model_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/mahout_model_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/openjpa_model_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/pig_model_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/tika_model_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/zookeeper_model_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/bookkeeper_corpus_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/mahout_corpus_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/openjpa_corpus_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/pig_corpus_sweep.tex
Writing to: /home/cscorley/git/dissertation/tables/tika_corpus_sweep.tex
Writing to: /home/cscorley/git/diss

In [42]:
best_dits['model']['all subject systems'], best_flts['model']['all subject systems']

(Task alpha   eta    K       DIT       FLT
 35       5  auto  500  0.381787  0.410651,
 Task alpha   eta    K       DIT       FLT
 23       2  auto  500  0.371773  0.416203)

In [43]:
FIG_TEX="""
%% new fig
\\begin{figure}
    \\centering
    \\begin{subfigure}{.4\\textwidth}
        \\centering
        \\includegraphics[height=0.4\\textheight]{%s}
        \\caption{Including outliers}\\label{fig:combo:%s_outlier}
    \\end{subfigure}%%
    \\begin{subfigure}{.4\\textwidth}
        \\centering
        \\includegraphics[height=0.4\\textheight]{%s_no_outlier}
        \\caption{Excluding outliers}\\label{fig:combo:%s_no_outlier}
    \\end{subfigure}
\\caption[%s effectiveness measures of optimal and alternative %s configurations for %s]%%
{%s effectiveness measures of optimal ($MRR=%.4f$) and alternative ($MRR=%.4f$) %s configurations for %s}
\\label{fig:combo:%s}
\\end{figure}
"""

def plot_dataframe(df, each, name, kind):
    order = ["Optimal", "Alternate"]
    
    kind = kind.lower()
    name = name.lower()
    
    size = (len(order)*1.6, 4.5)
    limitgrowth = 0.5
    fontsize = None
    widths = 0.3
    lower = 0
    kinds = {"flt": "Feature Location", "dit": "Developer Identification"}
    rqs = {"flt": {"rq1": "\\cone", "rq2": "\\ctwo", "all": "Overview"},
           "dit": {"rq1": "\\cone", "rq2": "\\ctwo", "all": "Overview"}}
    config_name = "model" if name == "rq1" else "corpus"
    
    result = df.plot(kind='box',
                     fontsize=fontsize,
                     figsize=size,
                     widths=widths,
                     y=order)
    
    limit = result.get_ylim()
    lower = limit[0] - limitgrowth
    if (lower < 0):
        lower = 0
        
    result.set_ylim(lower, limit[1] + limitgrowth)
    
    plt.tight_layout()
    short_each = each.lower().split(' ')[0]
    fig_name = 'figures/combo/%s_%s_%s' % (kind, name, short_each)
    path = os.path.expanduser('~/git/dissertation/') + fig_name
    plt.savefig(path + ".pdf", dpi=300)

    optimal_data = df["Optimal"].dropna()
    alternate_data = df["Alternate"].dropna()
    optimal_mrr = src.utils.calculate_mrr(optimal_data)
    alternative_mrr = src.utils.calculate_mrr(alternate_data)
    #T, p, r = wilcoxon(optimal_data, alternate_data)

    with open(path + ".tex", "wt") as f:
        figlabel = ":".join([x.lower() for x in [kind, name, short_each]])
        f.write(FIG_TEX % (fig_name, figlabel,
                           fig_name, figlabel,
                           kinds[kind], config_name, each, # toc caption
                           kinds[kind], # full figure caption
                           optimal_mrr, alternative_mrr,
                           config_name, each,
                           figlabel))
    plt.close()
    
    # no outliers              
    result = df.plot(kind='box',
         fontsize=fontsize,
         figsize=size,
         widths=widths,
         y=order,
         showfliers=False)
    limit = result.get_ylim()
    lower = limit[0] - limitgrowth
    if (lower < 0):
        lower = 0

    result.set_ylim(lower, limit[1] + limitgrowth)

    plt.tight_layout()

    fig_name = 'figures/combo/%s_%s_%s_no_outlier' % (kind, name, short_each)
    path = os.path.expanduser('~/git/dissertation/') + fig_name
    plt.savefig(path + ".pdf", dpi=300)
    plt.close()

In [63]:
corpus_wilcoxon_df = pandas.DataFrame(columns=["Subject", "Task",
                                               "Optimal Configuration", "Alternate Configuration",
                                               "Optimal MRR", "Alternate MRR",
                                               #"T",
                                               "p-value", "Effect size"])
model_wilcoxon_df = pandas.DataFrame(columns=["Subject", "Task",
                                              "Optimal Configuration", "Alternate Configuration",
                                              "Optimal MRR", "Alternate MRR",
                                              #"T",
                                              "p-value", "Effect size"])

for task, best_df, best_alt_df in [("FLT", best_flts, best_dits), ("DIT", best_dits, best_flts)]:
    for project, table in best_df['corpus'].items():
        alt_table = best_alt_df['corpus'][project]
        
        optimal_config = "$(%s)$" % ", ".join([key[0] for key, value in table.iteritems() if key != "FLT" and key != "DIT" and key != "Configuration" and value.values[0]])
        alt_config = "$(%s)$" % ", ".join([key[0] for key, value in alt_table.iteritems() if key != "FLT" and key != "DIT" and key != "Configuration" and value.values[0]])

        print(project, task, 'corpus', optimal_config, alt_config)
        #print(table.sample(1))
        optimal = corpus_df[
                        (corpus_df.Task == task) &
                        (corpus_df.Additions == table.Additions.values[0]) &
                        (corpus_df.Removals == table.Removals.values[0]) &
                        (corpus_df.Context == table.Context.values[0]) &
                        (corpus_df.Message == table.Message.values[0])]

        #print(alt_table.sample(1))

        alternate = corpus_df[
                              (corpus_df.Task == task) &
                              (corpus_df.Additions == alt_table.Additions.values[0]) &
                              (corpus_df.Removals == alt_table.Removals.values[0]) &
                              (corpus_df.Context == alt_table.Context.values[0]) &
                              (corpus_df.Message == alt_table.Message.values[0])]

        if project != "all subject systems":
            optimal = optimal[optimal.Subject == project]
            alternate = alternate[alternate.Subject == project]

        merge_df = optimal.merge(alternate, on=["Subject", "Task", "Issue"])
        optalt_df = pandas.DataFrame()
        optalt_df["Optimal"] = merge_df.Rank_x
        optalt_df["Alternate"] = merge_df.Rank_y

        plot_dataframe(optalt_df, project,  "rq2", task)
        
        if((optalt_df.Optimal == optalt_df.Alternate).all()):
            corpus_wilcoxon_df = corpus_wilcoxon_df.append({
                    "Subject": project,
                    "Task": task,
                    "Optimal MRR": numpy.mean(1/optalt_df.Optimal), 
                    "Alternate MRR": numpy.mean(1/optalt_df.Alternate),
                    "Optimal Configuration": optimal_config,
                    "Alternate Configuration": alt_config,
                    #"T": numpy.NaN,
                    "p-value": numpy.NaN,
                    "Effect size": numpy.NaN
                    }, ignore_index=True)
        else:
            T, p, r = wilcoxon(optalt_df.Optimal, optalt_df.Alternate)
            corpus_wilcoxon_df = corpus_wilcoxon_df.append({
                    "Subject": project,
                    "Task": task,
                    "Optimal MRR": numpy.mean(1/optalt_df.Optimal), 
                    "Alternate MRR": numpy.mean(1/optalt_df.Alternate),
                    "Optimal Configuration": optimal_config,
                    "Alternate Configuration": alt_config,
                    #"T": T,
                    "p-value": p,
                    "Effect size": r                   
                    }, ignore_index=True)
                               
for task, best_df, best_alt_df in [("FLT", best_flts, best_dits), ("DIT", best_dits, best_flts)]:
    for project, table in best_df['model'].items():
        alt_table = best_alt_df['model'][project]

        optimal_config = "$(%s)$" % ", ".join(["%s=%s" % (key, int(value.values[0]) if key == "K" else "%s/K" % value.values[0] if value.values[0] != "auto" else value.values[0]) for key, value in sorted(table.iteritems()) if key != "FLT" and key != "DIT" and key != "Configuration"]).replace("alpha", "\\alpha").replace("eta", "\\eta")
        alt_config = "$(%s)$" % ", ".join(["%s=%s" % (key, int(value.values[0]) if key == "K" else "%s/K" % value.values[0] if value.values[0] != "auto" else value.values[0]) for key, value in sorted(alt_table.iteritems()) if key != "FLT" and key != "DIT" and key != "Configuration"]).replace("alpha", "\\alpha").replace("eta", "\\eta")

        print(project, task, 'model', optimal_config, alt_config)
        #print(table.sample(1))
        optimal = model_df[
                        (model_df.Task == task) &
                        (model_df.alpha == table.alpha.values[0]) &
                        (model_df.eta == table.eta.values[0]) &
                        (model_df.K == table.K.values[0])]

        #print(alt_table.sample(1))

        alternate = model_df[
                        (model_df.Task == task) &
                        (model_df.alpha == alt_table.alpha.values[0]) &
                        (model_df.eta == alt_table.eta.values[0]) &
                        (model_df.K == alt_table.K.values[0])]

        if project != "all subject systems":
            optimal = optimal[optimal.Subject == project]
            alternate = alternate[alternate.Subject == project]

        merge_df = optimal.merge(alternate, on=["Subject", "Task", "Issue"])
        optalt_df = pandas.DataFrame()
        optalt_df["Optimal"] = merge_df.Rank_x
        optalt_df["Alternate"] = merge_df.Rank_y

        plot_dataframe(optalt_df, project, "rq1", task)
        if((optalt_df.Optimal == optalt_df.Alternate).all()):
            model_wilcoxon_df = model_wilcoxon_df.append({
                    "Subject": project,
                    "Task": task,
                    "Optimal MRR": numpy.mean(1/optalt_df.Optimal), 
                    "Alternate MRR": numpy.mean(1/optalt_df.Alternate),
                    "Optimal Configuration": optimal_config,
                    "Alternate Configuration": alt_config,
                    #"T": numpy.NaN,
                    "p-value": numpy.NaN,
                    "Effect size": numpy.NaN
                    }, ignore_index=True)
        else:
            T, p, r = wilcoxon(optalt_df.Optimal, optalt_df.Alternate)
            model_wilcoxon_df = model_wilcoxon_df.append({
                    "Subject": project,
                    "Task": task,
                    "Optimal MRR": numpy.mean(1/optalt_df.Optimal), 
                    "Alternate MRR": numpy.mean(1/optalt_df.Alternate),
                    "Optimal Configuration": optimal_config,
                    "Alternate Configuration": alt_config,
                    #"T": T,
                    "p-value": p,
                    "Effect size" : r                   
                    }, ignore_index=True)


Tika v1.8 FLT corpus $(A, R, C, M)$ $(A, C)$
OpenJPA v2.3.0 FLT corpus $(A, C, M)$ $(A, C)$
ZooKeeper v3.5.0 FLT corpus $(A, C, M)$ $(C)$
all subject systems FLT corpus $(A, C, M)$ $(C, M)$
Mahout v0.10.0 FLT corpus $(C, M)$ $(A)$
BookKeeper v4.3.0 FLT corpus $(A, C, M)$ $(A, R, C, M)$
Pig v0.14.0 FLT corpus $(A, R)$ $(C)$
Tika v1.8 DIT corpus $(A, C)$ $(A, R, C, M)$
OpenJPA v2.3.0 DIT corpus $(A, C)$ $(A, C, M)$
ZooKeeper v3.5.0 DIT corpus $(C)$ $(A, C, M)$
all subject systems DIT corpus $(C, M)$ $(A, C, M)$
Mahout v0.10.0 DIT corpus $(A)$ $(C, M)$
BookKeeper v4.3.0 DIT corpus $(A, R, C, M)$ $(A, C, M)$
Pig v0.14.0 DIT corpus $(C)$ $(A, R)$
Tika v1.8 FLT model $(K=200, \alpha=2/K, \eta=2/K)$ $(K=500, \alpha=2/K, \eta=5/K)$
OpenJPA v2.3.0 FLT model $(K=500, \alpha=2/K, \eta=auto)$ $(K=500, \alpha=auto, \eta=1/K)$
ZooKeeper v3.5.0 FLT model $(K=500, \alpha=1/K, \eta=auto)$ $(K=500, \alpha=2/K, \eta=2/K)$
all subject systems FLT model $(K=500, \alpha=2/K, \eta=auto)$ $(K=500, \alpha=5/K,

In [64]:
bon

0.01

In [65]:
def print_wilcox(df, task, kind):
    WILCOX="""
%% new fig
\\begin{table}
\\begin{spacing}{1.2}
\\centering
\\caption{Wilcoxon test results for %s optimal and alternative %s configurations}
\\label{table:combo-%s-%s-sweep-wilcox}
%s
\\end{spacing}
\\end{table}
"""
    path = os.path.expanduser('~/git/dissertation/tables/')
    path += "landscape_%s_sweep_wilcox.tex" % (kind.lower())
    with open(path, 'wt') as f:
        print("\\begin{landscape}", file=f)
        
    for task in ["FLT", "DIT"]:
        j = df[df.Task == task]
        del j["Task"]
        j = j.sort(columns=["Subject"])
        
        with open(path, 'at') as f:
            print(WILCOX % (task.upper(), kind.lower(), task.lower(), kind.lower(),
                            j.to_latex(index=False,
                                       escape=False, # needed so it doesn't screw up formatters
                                       formatters=formatters)
                             .replace('nan', '')
                             .replace('Optimal MRR & ', '')
                             .replace('Alternate MRR', '\\multicolumn{2}{c}{MRRs} ')), file=f)
    with open(path, 'at') as f:
        print("\\end{landscape}", file=f)

In [66]:
print_wilcox(model_wilcoxon_df, "FLT", "model")

In [67]:
print_wilcox(model_wilcoxon_df, "DIT", "model")

In [68]:
print_wilcox(corpus_wilcoxon_df, "FLT", "corpus")

In [69]:
print_wilcox(corpus_wilcoxon_df, "DIT", "corpus")

In [70]:
corpus_wilcoxon_df["Optimal Configuration"].str.contains('A').value_counts()

True     10
False     4
dtype: int64

In [71]:
corpus_wilcoxon_df["Optimal Configuration"].str.contains('C').value_counts()

True     12
False     2
dtype: int64

In [72]:
corpus_wilcoxon_df["Optimal Configuration"].str.contains('M').value_counts()

True     8
False    6
dtype: int64

In [73]:
corpus_wilcoxon_df["Optimal Configuration"].str.contains('R').value_counts()

False    11
True      3
dtype: int64

In [74]:
versus = dict()
for kind in 'ACMR':
    versus_r = cres[(cres.Config.str.contains(kind) & ~cres.Config2.str.contains(kind) & (cres.Config.str.replace(kind, '') == cres.Config2))]

    # Includes, Excludes
    versus[kind] = {'FLT Included': versus_r[(versus_r.MRR > versus_r.MRR2) & (versus_r.Task == "FLT")].Subject.value_counts(),
                    'FLT Excluded': versus_r[(versus_r.MRR2 > versus_r.MRR) & (versus_r.Task == "FLT")].Subject.value_counts(),
                    'DIT Included': versus_r[(versus_r.MRR > versus_r.MRR2) & (versus_r.Task == "DIT")].Subject.value_counts(),
                    'DIT Excluded': versus_r[(versus_r.MRR2 > versus_r.MRR) & (versus_r.Task == "DIT")].Subject.value_counts(),
                    'Both Included': versus_r[(versus_r.MRR > versus_r.MRR2)].Subject.value_counts(),
                    'Both Excluded': versus_r[(versus_r.MRR2 > versus_r.MRR)].Subject.value_counts(),
                   }

versus = pandas.Panel(versus).fillna(0)

In [75]:
versus['A']

,Both Excluded,Both Included,DIT Excluded,DIT Included,FLT Excluded,FLT Included
BookKeeper v4.3.0,5,9,5,2,0,7
Mahout v0.10.0,6,8,2,5,4,3
OpenJPA v2.3.0,0,14,0,7,0,7
Pig v0.14.0,6,8,4,3,2,5
Tika v1.8,4,10,3,4,1,6
ZooKeeper v3.5.0,3,11,2,5,1,6
all subject systems,2,12,2,5,0,7


In [76]:
versus['C']

,Both Excluded,Both Included,DIT Excluded,DIT Included,FLT Excluded,FLT Included
BookKeeper v4.3.0,3,11,0,7,3,4
Mahout v0.10.0,4,10,4,3,0,7
OpenJPA v2.3.0,5,9,1,6,4,3
Pig v0.14.0,5,9,2,5,3,4
Tika v1.8,5,9,1,6,4,3
ZooKeeper v3.5.0,2,12,1,6,1,6
all subject systems,2,12,0,7,2,5


In [77]:
versus['M']

,Both Excluded,Both Included,DIT Excluded,DIT Included,FLT Excluded,FLT Included
BookKeeper v4.3.0,0,14,0,7,0,7
Mahout v0.10.0,7,7,6,1,1,6
OpenJPA v2.3.0,5,9,2,5,3,4
Pig v0.14.0,6,8,3,4,3,4
Tika v1.8,6,8,3,4,3,4
ZooKeeper v3.5.0,7,7,5,2,2,5
all subject systems,2,12,1,6,1,6


In [78]:
versus['R']

,Both Excluded,Both Included,DIT Excluded,DIT Included,FLT Excluded,FLT Included
BookKeeper v4.3.0,6,8,4,3,2,5
Mahout v0.10.0,10,4,6,1,4,3
OpenJPA v2.3.0,6,8,2,5,4,3
Pig v0.14.0,9,5,5,2,4,3
Tika v1.8,10,4,5,2,5,2
ZooKeeper v3.5.0,7,7,3,4,4,3
all subject systems,8,6,5,2,3,4


In [79]:
versus_wilcox = dict()
for kind in 'ACMR':
    versus_r = cres[(cres.Config.str.contains(kind) & ~cres.Config2.str.contains(kind) & (cres.Config.str.replace(kind, '') == cres.Config2))]
    del(versus_r["T"])
    versus_wilcox[kind] = versus_r

    #versus_wilcox["Config"] = versus_wilcox["Config"].apply(lambda x: "$(%s)$" % ", ".join(x))

versus_wilcox = pandas.Panel(versus_wilcox)
versus_wilcox["A"].dropna()

,Subject,Task,Config,Config2,MRR,MRR2,p,r
7,BookKeeper v4.3.0,DIT,ACMR,CMR,0.7216463,0.6530488,2.652274e-06,0.5688098
21,BookKeeper v4.3.0,DIT,ACM,CM,0.6973577,0.7113821,0.0002447951,0.4426624
34,BookKeeper v4.3.0,DIT,ACR,CR,0.6513211,0.6526423,0.4967107,0.08169262
46,BookKeeper v4.3.0,DIT,AC,C,0.6517276,0.7056911,5.973301e-06,0.5177782
57,BookKeeper v4.3.0,DIT,AMR,MR,0.6465447,0.6507114,0.3266543,0.1128589
67,BookKeeper v4.3.0,DIT,AM,M,0.6721545,0.5132114,0.01720218,0.2335512
76,BookKeeper v4.3.0,DIT,AR,R,0.5424797,0.6375,0.0002120283,0.4315789
112,BookKeeper v4.3.0,FLT,ACMR,CMR,0.5245999,0.4786202,0.1602487,0.1586081
126,BookKeeper v4.3.0,FLT,ACM,CM,0.532703,0.496799,0.1358118,0.1622946
139,BookKeeper v4.3.0,FLT,ACR,CR,0.4566713,0.4035509,0.03983517,0.2208396


In [81]:
TEX="""
\\begin{table}
\\begin{spacing}{1.2}
\\centering
\\caption{Wilcoxon test results for %s inclusion and exclusion configurations of the %s task for %s}
\\label{table:%s}
%s
\\end{spacing}
\\end{table}
"""

kinds = ["additions", "removals", "context", "message"]

def reorg_string(s):
    new = ""
    s = s.upper()
    for each in 'ARCM':
        if each in s:
            new += each 
    return new
 
float_format = lambda x: "$%1.4f$" % x
float_format_bold = lambda x: "$\\bm{%1.4f}$" % x

fasdf = {
    "p-value": lambda x: "$p<0.01$" if x < 0.01 else float_format(x) ,
    "Effect size": float_format,
    "Config": lambda x: "$(%s)$" % ','.join(reorg_string(x)),
    "Config2": lambda x: "$(%s)$" % ','.join(reorg_string(x)),
}

    
for source in ["additions", "removals", "context", "message"]:
    df = versus_wilcox[source[0].upper()].dropna()
    for key, group in df.groupby(["Subject", "Task"]):
        subject, task = key
        a = pandas.DataFrame(group) # drops warnings for http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

        a["p-value"] = a["p"].astype(float)
        a["Effect size"] = a["r"].astype(float)
        a["MRR"] = a["MRR"].astype(float)
        a["MRR2"] = a["MRR2"].astype(float)
        del[a["p"]]
        del[a["r"]]
        del[a["Subject"]]
        del[a["Task"]]
        a = pandas.DataFrame(a)
        
        left = (a.MRR > a.MRR2)
        right = (a.MRR <= a.MRR2)
        a.MRR.update(a[left].MRR.apply(float_format_bold))
        a.MRR.update(a[right].MRR.apply(float_format))
        a.MRR2.update(a[left].MRR2.apply(float_format))
        a.MRR2.update(a[right].MRR2.apply(float_format_bold))
        

        s = subject.split()[0].lower()
        name = "versus-wilcox-%s-%s-%s" % (s, task.lower(), source)
        
        path = os.path.expanduser('~/git/dissertation/tables/')
        path += "%s.tex" % (name.lower())
        with open(path, 'wt') as f:
            print(TEX % (source, task.upper(), subject, name,
                  a.to_latex(index=False, escape=False, formatters=fasdf)
                   .replace("Config ", "\\multicolumn{2}{c|}{Configurations} ")
                   .replace("Config2 &", "")
                   .replace("MRR ", "\\multicolumn{2}{c|}{MRRs} ")
                   .replace("MRR2 &", "")
                   .replace("llllrr", "ll|rr|rr")),
                 file=f)